# Archived
**We are no longer running ETLs by running this notebook. This work has been transitioned to [Product Analytics job](https://gerrit.wikimedia.org/r/plugins/gitiles/analytics/wmf-product/jobs/+/refs/heads/master/movement_metrics/)**

In [1]:
from pathlib import Path
import datetime
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from wmfdata import charting, hive

# Parameters

In [2]:
# UPDATE FOR NEW MONTH!
# Metric month. The mediawiki_history snapshot for this month must be available.
## METRICS_MONTH_TEXT = "2019-07"
## MEDIAWIKI_HISTORY_SNAPSHOT = "2019-07"

last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")

metrics_year =  last_month.year
metrics_cur_month = last_month.month


# Update table

## Monthly Pageviews ##

Pageviews data comes from wmf.pageview_hourly for spurious IE views from some countries. Corrected data is stored in intermediate hive table mneisler.pageviews_corrected with data stored back to September 2016. 

List of corrections and timespans that they are applied to:

* IEPKIRAF: since May 20, 2018
* IE7PKIRAF: January 1-May 19, 2018
* IE7PKIRAF & iOS mainpage: Sept 2016-Dec 31, 2017


In [3]:
update_pv = Path("queries/update_pageviews_table.sql").read_text().format(
    metrics_year=metrics_year,
    metrics_cur_month=metrics_cur_month
)

In [4]:
hive.run(["""
SET hive.exec.dynamic.partition.mode = 'nonstrict'""",update_pv])

,dayy,apps,desktop,mobileweb,total,year,month,day


In [5]:
#update database access
os.system("hdfs dfs -chmod -R o+r /user/hive/warehouse/wmf_product.db/pageviews_corrected")

0